In [77]:
import numpy as np
import pandas as pd
import os
import time
import glob
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import lr_scheduler

from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

from tools import EarlyStopping, adjust_learning_rate
import xLSTMTime

In [38]:
class Config():
    def __init__(self):
        self.seq_len = 9  # time length
        self.input_size = 10  # num_features
        self.target_len = 3
        self.head_size = 16
        self.num_heads = 2
        self.proj_factor_slstm = 4/3
        self.proj_factor_mlstm = 2
        self.layer_type = "msm"
        self.batch_first = True
        self.linear_embed_dim = 64
        self.split_ratio = 0.9
        self.batch_size = 32
        self.epochs = 500
        self.lr = 0.001
        self.pct_start = 0.3
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.lradj = 'TST'
        self.path = './results'
        self.patience = 100

configs = Config()

data

In [6]:
df = pd.read_csv('./dataset/train_target_assemble(18-21).csv')

In [8]:
# 시점 분리
df_data = df[df.columns[1:]]
df_stamp = df[df.columns[0]]

In [ ]:
train_size=int(len(df_data) * configs.split_ratio)
train_set,test_set=df_data.iloc[:train_size],df_data.iloc[train_size:]
print(f"train size={len(train_set)}\ntest_size={len(test_set)}")

In [ ]:
scaled_columns=train_set.columns
scaler=StandardScaler()
scaler=scaler.fit(train_set[scaled_columns].values)
train_set[scaled_columns] = scaler.transform(train_set[scaled_columns].values)
test_set[scaled_columns] = scaler.transform(test_set[scaled_columns].values)

In [19]:
def create_dataset(dataset, seq_len=8, target_len=8):
    dataX, dataY = [], []
    for i in range(len(dataset) - seq_len - target_len):
        x_data = dataset.iloc[i:(i + seq_len), :].values
        dataX.append(x_data)
        y_data = dataset.iloc[(i + seq_len):(i + seq_len)+target_len, :].values
        dataY.append(y_data)
    return torch.Tensor(dataX), torch.Tensor(dataY)

In [ ]:
X_train, y_train = create_dataset(train_set, configs.seq_len, configs.target_len)
X_test, y_test = create_dataset(test_set, configs.seq_len, configs.target_len)
print(X_train.shape, y_train.shape)

In [23]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=configs.batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=configs.batch_size, shuffle=False)

model

In [25]:
model = xLSTMTime.Model(configs)

In [ ]:
model.to(configs.device)

In [41]:
def validation(model, test_loader, configs):
    criterion = torch.nn.SmoothL1Loss()
    
    valid_losses = []
    model.eval()
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(test_loader):
            inputs = inputs.to(configs.device)
            targets = targets.to(configs.device)

            outputs = model(inputs)

            outputs = outputs.detach().cpu()
            targets = targets.detach().cpu()

            loss = criterion(outputs, targets)
            valid_losses.append(loss)
    
    valid_losses = np.average(valid_losses)
    model.train()
    return valid_losses

In [42]:
def train_model(model, train_loader, test_loader, configs):
    criterion = torch.nn.SmoothL1Loss()
    optimizer = torch.optim.Adam(model.parameters(), lr=configs.lr)
    scheduler = lr_scheduler.OneCycleLR(optimizer = optimizer,
                                            steps_per_epoch = len(train_loader),
                                            pct_start = configs.pct_start,
                                            epochs = configs.epochs,
                                            max_lr = configs.lr)
    early_stopping = EarlyStopping(patience=configs.patience, verbose=True)


    for epoch in tqdm(range(configs.epochs), desc=f'Training', colour='GREEN'):
        iter_count = 0
        train_losses = []
        model.train()
        for i, (inputs, targets) in enumerate(train_loader):
            iter_count += 1

            optimizer.zero_grad()
            inputs = inputs.to(configs.device)
            targets = targets.to(configs.device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            adjust_learning_rate(optimizer, scheduler, epoch + 1, configs, printout=False)
            scheduler.step()

            train_losses.append(loss.item())

            if (i + 1) % 100 == 0:
                print(f"\tepoch: {epoch+1} | iters: {i+1} | loss: {loss.item():.4f}")
                iter_count = 0

        train_losses = np.average(train_losses)
        valid_losses = validation(model, test_loader, configs)
        print(f"Epoch: {epoch+1}, Steps: {len(train_loader)} | Train Loss: {train_losses:.4f} Vali Loss: {valid_losses:.4f}") 
        early_stopping(valid_losses, model, configs.path)
        if early_stopping.early_stop:
            print("Early stopping")
            break
        
        adjust_learning_rate(optimizer, scheduler, epoch + 1, configs)

    best_model_path = configs.path + '/' + 'checkpoint.pth'
    model.load_state_dict(torch.load(best_model_path))

    return model

In [ ]:
trained_models = train_model(model, train_loader, test_loader, configs)

In [ ]:
# predict

test_files = glob.glob('./dataset/test_*_assemble.csv')

result_df = pd.DataFrame()

for i, files in tqdm(enumerate(test_files)):
    test_df = pd.read_csv(files)
    test_df = test_df[test_df.columns[1:]]
    
    scaled_test = scaler.transform(test_df.values)

    X_predict = torch.Tensor(scaled_test)
    pred_dataset = TensorDataset(X_predict)
    pred_loader = DataLoader(pred_dataset, batch_size=configs.batch_size, shuffle=False)


    trained_models.eval()
    with torch.no_grad():
        for inputs in pred_loader:
            inputs = inputs[0]
            inputs = inputs.unsqueeze(dim=0)
            inputs = inputs.to(configs.device)

            outputs = trained_models(inputs)

            outputs = outputs.detach().cpu()

    df = pd.DataFrame(outputs.squeeze(0).numpy())
    unscaled_df = pd.DataFrame(scaler.inverse_transform(df), columns=df_data.columns)
    result_df = pd.concat([result_df, unscaled_df], ignore_index=True)

In [ ]:
result_df

In [118]:
result_df.rename(columns={'깐마늘':'깐마늘(국산)'}, inplace=True)

In [130]:
submission = pd.read_csv('./dataset/sample_submission.csv')

In [ ]:
submission

In [132]:
sub_date = submission[submission.columns[0]]
sub_data = submission[submission.columns[1:]]

In [133]:
result_df = result_df[sub_data.columns]

In [ ]:
result_df

In [134]:
my_result = pd.concat([sub_date, result_df], axis=1)

In [ ]:
my_result

In [138]:
my_result.to_csv(f'sample_submission.csv', index=False, encoding="utf-8-sig")